In [1]:
import argparse
import pickle
from pathlib import Path

import fraud_eagle as feagle
import fraudar
import networkx as nx
import numpy as np
import pandas as pd
import rsd

from rev2 import rev2compute
from utils import load_data, split_data_by_time

import seaborn as sns
import matplotlib.pyplot as plt

import sklearn.metrics

import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings(action='ignore', category=UndefinedMetricWarning)

In [49]:
alg_list = ["fraudar", "rsd", "rev2", "sg"][:]
data_list = ["alpha", "otc", "amazon", "epinions"][2:3]
budget_list = [100.0, 200.0, 300.0, 400.0]
frac_list = [0.0, 0.2, 0.4, 0.6, 0.8]

In [17]:
def get_results(data_name="alpha", alg="fraudar", attack_name="ilp_attack", budget=100):
    data_nw_df, data_gt_df = load_data(data_name=data_name)
    p = Path(f"../res/{attack_name}/{alg}-{data_name}/{budget}.pkl")
    if not p.exists():
        print(p)
        return None
    with open(p, "rb") as fp:
        scores = pickle.load(fp)
        scores = dict(zip(range(len(scores)), scores))
        # 1 means fraudster, 0 means benign
        scores["truth"] = {
            a: dict(zip(data_gt_df["id"], (1-data_gt_df["label"])/2))
            for a in alg_list
        }
    # df = pd.DataFrame({sp: {u: scores[sp][u] for u in scores[sp] if u in data_gt_df["id"].tolist()} for sp in scores})
    df = pd.DataFrame({(a, sp): {u: scores[sp][a][u] for u in scores[sp][a]} for a in alg_list for sp in scores if a in scores[sp]})
    df.loc[[u[:5]=="usock" for u in df.index.tolist()], (slice(None), "truth")] = 1
    df.loc[[u[:6]=="udummy" for u in df.index.tolist()], (slice(None), "truth")] = 1
    return df

In [27]:
def get_metrics(df_res, splits=4, K=None, th_dict=None):
    if df_res is None:
        return None
    ret = {}
    for sp in range(splits):
        # df_res["truth"] = df_res["truth"].fillna(1)
        # df_res.loc[[u[:5]=="usock" for u in df_res.index.tolist()], "truth"] = 1
        # df_res.loc[[u[:6]=="udummy" for u in df_res.index.tolist()], "truth"] = 1
        for a in alg_list:
            if (a, sp) not in df_res.columns:
                continue
            df = df_res[[(a, sp), (a, "truth")]].dropna()
            t = df[(a, "truth")].sum() if K is None else K
            t_frac = t / df.shape[0]
            # df["pred"] = (df[sp].rank(ascending=False, method="min") <= t) * 1
            # df["pred"] = df[sp] >= t_frac
            df[(a, "pred")] = df[(a, sp)] >= th_dict[a]

            ret[(a, sp)] = {
                "precision": sklearn.metrics.precision_score(y_true=df[(a, "truth")], y_pred=df[(a, "pred")]),
                "recall": sklearn.metrics.recall_score(y_true=df[(a, "truth")], y_pred=df[(a, "pred")]),
                "f1": sklearn.metrics.f1_score(y_true=df[(a, "truth")], y_pred=df[(a, "pred")])
            }
    return ret

In [50]:
attack_res = {
    (d, a, b): get_results(data_name=d, alg=a, attack_name="sockfarm_attack", budget=b)
    for d in data_list for a in alg_list
    for b in budget_list
}

../res/sockfarm_attack/sg-amazon/300.0.pkl
../res/sockfarm_attack/sg-amazon/400.0.pkl


In [10]:
# attack_res[("amazon", "rev2", 400.0)].to_csv("ar.csv")

In [51]:
def get_sockfarm_alg_data_met(df_met, data="alpha", met="f1"):
    rdf = df_met.reset_index().pivot(index=["data", "algo", "metrics"], columns=["budget"]).loc[(data, slice(None), met)].droplevel([0, 2], axis=0)
    rdf.index = [f"sockfarm-{i}" for i in rdf.index]
    return rdf

In [52]:
def_dict = {
    "fraudar": {"fraudar": 0.9, "rsd": 1.96, "rev2": 0.00, "sg": 0.5}, # rev2=0.026
    "rsd": {"fraudar": 0.5, "rsd": 0.6, "rev2": 0.09, "sg": 0.5},
    "rev2": {"fraudar": 0.9, "rsd": 1.92, "rev2": 0.5, "sg": 0.5},
    "sg": {"fraudar": 0.9, "rsd": 1.4, "rev2": 0.15, "sg": 0.5},
}


dict_met = {
    (d, a, b): pd.DataFrame.from_dict(get_metrics(df_res=attack_res[(d, a, b)], splits=4, K=None, th_dict=def_dict[a]))
    for d, a, b in attack_res
}
df_met = pd.concat(dict_met, keys=dict_met.keys())
df_met.index = df_met.index.set_names(["data", "algo", "budget", "metrics"])
df_met = df_met.mean(axis=1, level=0)
# display(df_met)

dmet="amazon"

display(get_sockfarm_alg_data_met(df_met, data=dmet, met="f1"))
display(get_sockfarm_alg_data_met(df_met, data=dmet, met="recall"))
display(get_sockfarm_alg_data_met(df_met, data=dmet, met="precision"))

IndexError: Too many levels: Index has only 1 level, not 3